#  FHIR Data Simulation from Framingham Dataset
This notebook converts sample Framingham-style cardiovascular data into FHIR-compliant Observation JSON files.

In [16]:
import os
import json
import pandas as pd
from datetime import datetime

In [17]:
# Load Framingham data from user's path (assumed content structure)
framingham_csv = "../data/framingham.csv"
df = pd.read_csv(framingham_csv)


In [18]:
# Use a subset for demo
df = df[["age", "sysBP", "diaBP", "totChol", "glucose", "heartRate"]].head(3)

# LOINC mappings
loinc_map = {
    "age": ("30525-0", "Age calculated", "years"),
    "sysBP": ("8480-6", "Systolic blood pressure", "mmHg"),
    "diaBP": ("8462-4", "Diastolic blood pressure", "mmHg"),
    "totChol": ("2093-3", "Total Cholesterol", "mg/dL"),
    "glucose": ("2345-7", "Glucose", "mg/dL"),
    "heartRate": ("8867-4", "Heart rate", "beats/minute")
}


In [22]:
# Output directory
output_dir = "../data/observations"
os.makedirs(output_dir, exist_ok=True)

In [24]:
# Convert to FHIR JSON manually
for idx, row in df.iterrows():
    patient_id = f"Patient-{1000 + idx}"
    for feature, value in row.items():
        if pd.isna(value): continue
        code, display, unit = loinc_map[feature]

        obs = {
            "resourceType": "Observation",
            "id": f"{patient_id}-{feature}",
            "status": "final",
            "code": {
                "coding": [{
                    "system": "http://loinc.org",
                    "code": code,
                    "display": display
                }],
                "text": display
            },
            "subject": {
                "reference": patient_id
            },
            "effectiveDateTime": datetime.utcnow().isoformat(),
            "valueQuantity": {
                "value": float(value),
                "unit": unit,
                "system": "http://unitsofmeasure.org",
                "code": unit
            }
        }

        out_path = os.path.join(output_dir, f"{obs['id']}.json")
        with open(out_path, "w") as f:
            json.dump(obs, f, indent=2)

output_dir


C:\Users\tranv\AppData\Local\Temp\ipykernel_6320\3928600074.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "effectiveDateTime": datetime.utcnow().isoformat(),


'../data/observations'